## 환경설정

1. http://braincrew2.iptime.org:8001 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 기입하시는 것을 권장 드려요. 예. 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.


In [ ]:
project = "ADVHOUSE"  # 수정하지 마세요
username = "g4312@naver.com"  # 이메일아이디 (예시. abc@hello.com
password = "1234"  # 비밀번호

아래의 코드를 순서대로 실행해 주세요.


In [ ]:
import os
import requests

if not os.path.exists("competition.py"):
    url = "https://link.teddynote.com/COMPT"
    file_name = "competition.py"
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

아래 코드를 실행하여 데이터를 다운로드 받습니다.


In [ ]:
import competition

# 파일 다운로드
# competition.download_competition_files(project)
competition.download_competition_files(
    "https://link.teddynote.com/ADVHOUSE", use_competition_url=False
)

100%|██████████| 164k/164k [00:00<00:00, 6.25MiB/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# Data 경로 설정
DATA_DIR = "data"

# 경고 무시
warnings.filterwarnings("ignore")

SEED = 123

## 데이터 로드


In [ ]:
# train 데이터셋 로드 (train.csv)
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))

# test 데이터셋 로드 (test.csv)
test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### 기본 전처리


In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [ ]:
numeric_columns = all_data.select_dtypes(exclude="object") #문자열 처리
numeric_columns

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,61,0,0,0,0,0,2,2008,208500.0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,298,0,0,0,0,0,0,5,2007,181500.0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,42,0,0,0,0,0,9,2008,223500.0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,35,272,0,0,0,0,2,2006,140000.0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,192,84,0,0,0,0,0,12,2008,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,0,0,0,0,6,2006,NaN
2915,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,24,0,0,0,0,0,4,2006,NaN
2916,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,474,0,0,0,0,0,0,9,2006,NaN
2917,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,80,32,0,0,0,0,700,7,2006,NaN


In [ ]:
numeric_columns = numeric_columns.fillna(0)
numeric_columns

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,61,0,0,0,0,0,2,2008,208500.0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,298,0,0,0,0,0,0,5,2007,181500.0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,42,0,0,0,0,0,9,2008,223500.0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,35,272,0,0,0,0,2,2006,140000.0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,192,84,0,0,0,0,0,12,2008,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,0,0,0,0,6,2006,0.0
2915,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,24,0,0,0,0,0,4,2006,0.0
2916,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,474,0,0,0,0,0,0,9,2006,0.0
2917,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,80,32,0,0,0,0,700,7,2006,0.0


In [ ]:
numeric_columns = numeric_columns.drop('Id',axis=1)

In [ ]:
train_data = numeric_columns[: len(train)]
test_data = numeric_columns[len(train):]

### 데이터셋 분할


In [ ]:
x = train_data.drop("SalePrice", axis=1)
y = train_data["SalePrice"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y)

In [ ]:
x_test = test_data.drop("SalePrice", axis=1)

### CustomDataset 생성


In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


class CustomDataset(Dataset):
    def __init__(self, x, y=None, normalize=True):
        super(CustomDataset, self).__init__()
        self.x = x
        self.y = y

        # 데이터 표준화
        if normalize:
            scaler = StandardScaler()
            self.x = pd.DataFrame(scaler.fit_transform(self.x))

        # 텐서 변환
        self.x = torch.tensor(self.x.values).float()

        if y is not None:
            self.y = torch.tensor(self.y).float()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        if self.y is not None:
            y = self.y[idx]
            return x, y
        else:
            return x

### Dataset 생성


In [ ]:
y_train

840    140000.0
679    128500.0
899    135000.0
53     385000.0
532    107500.0
         ...   
111    180000.0
164    152000.0
897    142953.0
194    127000.0
126    128000.0
Name: SalePrice, Length: 1095, dtype: float64

In [ ]:
train_dataset = CustomDataset(x_train,y_train,normalize=True)
test_dataset = CustomDataset(x_test)
valid_dataset = CustomDataset(x_valid,y_valid)

KeyError: 3

### Data Loader 생성


In [ ]:
from torch.utils.data import DataLoader

# batch_size = 32 로 설정하고, shuffle을 True로 설정
train_loader = DataLoader(train_dataset, batch_size=32,shuffle=True)

# batch_size = 1 로 설정하고, shuffle을 False로 설정
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False)

### GPU 설정


In [ ]:
# Device 설정 (cuda:0 혹은 cpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### 모델 정의


In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# sequentail 쓰자
class Net(nn.Module):
    def __init__(self, num_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_features, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc3 = nn.Linear(24, 12)
        self.fc4 = nn.Linear(12, 8)
        # 마지막 출력층의 Neuron은 1개로 설정
        self.output = nn.Linear(8, 1)

    def forward(self, x):
        # 코드 입력
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))

        x = self.output(x)

        return x

In [ ]:
# 컬럼수 정의
NUM_FEATURES = len(x_train.columns)
print(NUM_FEATURES)

36


### 모델 생성


In [ ]:
# 모델 생성
model = Net(NUM_FEATURES)
# 모델을 device 에 올립니다. (cuda:0 혹은 cpu)
model.to(device)
model

Net(
  (fc1): Linear(in_features=36, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=12, bias=True)
  (fc4): Linear(in_features=12, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=1, bias=True)
)

### 손실함수와 옵티마이저 정의


In [ ]:
# Mean Squared Error(MSE) 오차 정의
loss_fn = nn.MSELoss()

In [ ]:
# 옵티마이저 설정: model.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=0.05)

### 학습(Training)


In [ ]:
# 최대 반복 횟수 정의
num_epoch = 500

# loss 기록하기 위한 list 정의
losses = []

for epoch in range(num_epoch):
    # loss 초기화
    running_loss = 0
    for x, y in train_loader:
        # x, y 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        x = x.to(device)
        y = y.to(device)

        # 그라디언트 초기화 (초기화를 수행하지 않으면 계산된 그라디언트는 누적됩니다.)
        optimizer.zero_grad()

        # output 계산
        y_hat = model(x)


        # 손실(loss) 계산
        loss = loss_fn(y.view(-1,1),y_hat)


        # 미분 계산
        loss.backward()


        # 경사하강법 계산 및 적용
        optimizer.step()


        # 배치별 loss 를 누적합산 합니다.
        running_loss += loss.item()

    # 누적합산된 배치별 loss값을 배치의 개수로 나누어 Epoch당 loss를 산출합니다.
    loss = running_loss / len(train_loader)
    losses.append(loss)

    # 20번의 Epcoh당 출력합니다.
    if epoch % 20 == 0:
        print("{0:05d} loss = {1:.5f}".format(epoch, loss))

print("----" * 15)
print("{0:05d} loss = {1:.5f}".format(epoch, loss))

00000 loss = 23520663084.52174
00020 loss = 761001536.00000
00040 loss = 735070456.17391
00060 loss = 624014030.26087
00080 loss = 613965905.04348
00100 loss = 640850784.34783
00120 loss = 735676193.04348
00140 loss = 610021408.00000
00160 loss = 617498785.04348
00180 loss = 514477946.08696
00200 loss = 605625288.00000
00220 loss = 485551665.21739
00240 loss = 565903507.82609
00260 loss = 452892388.69565
00280 loss = 441460883.13043
00300 loss = 452448871.47826
00320 loss = 433715612.17391
00340 loss = 391463897.39130
00360 loss = 356095012.00000
00380 loss = 278864897.04348
00400 loss = 278172392.86957
00420 loss = 209546979.65217
00440 loss = 250710801.56522
00460 loss = 266959177.73913
00480 loss = 216156512.17391
------------------------------------------------------------
00499 loss = 180530308.52174


### 예측 코드


In [ ]:
# 결과입력
your_answer = []

# evaluation mode 로 변경
model.eval()

with torch.no_grad():
    for x in test_loader:
        x = x.to(device)
        y_hat = model(x).cpu().detach().numpy().item()
        your_answer.append(y_hat)

## 결과 제출

- 느리다고 중지 후 다시 평가 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능)
- 제출결과는 [대회페이지](http://braincrew2.iptime.org:8001/competitions/ADVHOUSE/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.


아래 Cell을 실행하여 예측 결과 업데이트


In [ ]:
import competition

# 예측 결과 업데이트
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["SalePrice"] = your_answer

display(submission)
competition.submit(project, username, password, submission)

,Id,SalePrice
0,1461,113581.703125
1,1462,118782.023438
2,1463,188936.515625
3,1464,193262.406250
4,1465,178519.000000
...,...,...
1454,2915,113696.257812
1455,2916,78138.281250
1456,2917,209793.109375
1457,2918,70394.429688


아이디:  g4312@naver.com
파일명:  submissions/20240412-024103-submission.csv
[제출에 성공하였습니다]
제출 결과: 33380.91649950426
